# Exploration Data Analysis & Pre-Processing

## Preliminary Operations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import ountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.width', 1000) 
#pd.set_option('display.max_colwidth', None)

In [ ]:
# Trattamento dei valori mancanti
#df['claim'] = df['claim'].fillna('')  # Riempie i valori mancanti con una stringa vuota->potrebbe servire se non si applica la rimozione della punteggiatura

# Converti tutte le frasi in minuscolo
#df['claim'] = df['claim'].str.lower()

# Rimuovi la punteggiatura
#df['claim'] = df['claim'].apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)) if isinstance(x, str) else x)

# Tokenization
#df['claim'] = df['claim'].apply(lambda x: word_tokenize(x) if isinstance(x, str) else x)

# Rimuovi le stop words
#stop_words = set(stopwords.words('english'))
#df['claim'] = df['claim'].apply(lambda x: [word for word in x if word not in stop_words] if isinstance(x, list) else x)

# Lemmatizzazione
#lemmatizer = WordNetLemmatizer()
#df['claim'] = df['claim'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x] if isinstance(x, list) else x)

# Stemming
#ps = PorterStemmer()
#df['claim'] = df['claim'].apply(lambda x: [ps.stem(word) for word in x])

## PubHealth Dataset


### Train Dataset

In [ ]:
df_train = pd.read_csv(r'Thesis\PUBHEALTH\train.tsv', sep='\t')
df_train

In [ ]:
df_train[['claim_id','claim'.'explanation']].info()

In [ ]:
print("Valori unici in 'label':")
print(df_train['label'].nunique())
print(df_train['label'].unique())
# Conta le righe con label 'true'
true_label_count = df_train[df_train['label'] == 'true'].shape[0]
print(f"Numero di righe con label 'true': {true_label_count}")


In [ ]:
print("Valori unici in 'subjects':")
print(df_train['subjects'].nunique())
print(df_train['subjects'].unique())

In [ ]:
# Prima, convertiamo la colonna 'subjects' in una serie di liste di subjects
subjects_series = df_train['subjects'].dropna().str.split(',')

# Ora, "esplodiamo" questa serie in modo che ogni elemento di ogni lista diventi una riga
exploded_subjects = subjects_series.explode()

# Calcoliamo la frequenza di ogni subject unico
subject_frequencies = exploded_subjects.value_counts()

# Stampiamo le frequenze
print(subject_frequencies)

In [ ]:
# Rimuovere gli spazi bianchi all'inizio e alla fine di ogni stringa e dividere per virgola
subjects_series = df_train['subjects'].dropna().str.strip().str.split(',')

# Pulire ogni elemento all'interno delle liste rimuovendo spazi extra
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])

# Aggiungere un passaggio per sostituire le liste vuote con un valore che indica l'assenza di soggetti
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)

# "Esplodere" la serie per avere ogni soggetto come una riga separata
exploded_subjects = subjects_series.explode()

# Se ci sono state liste vuote, ora saranno NaN, quindi sostituire NaN con un placeholder
# Se vuoi contare questi come una categoria separata, puoi sostituirli con una stringa specifica
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare le frequenze, inclusi i valori placeholder per le righe vuote
subject_frequencies = exploded_subjects.value_counts()

# Stampare le frequenze, che ora includono anche il conteggio per 'No Subject'
print(subject_frequencies)
print(len(subject_frequencies))



In [ ]:
# Pulizia iniziale dei dati, inclusa la gestione dei soggetti vuoti
subjects_series = df_train['subjects'].dropna().str.strip().str.split(',')
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)
exploded_subjects = subjects_series.explode()
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare la frequenza di ciascun soggetto, inclusi quelli etichettati come 'No Subject'
subject_counts = exploded_subjects.value_counts()

# Selezionare i primi 20 soggetti più frequenti
top_subjects = subject_counts.head(20)

# Normalizzazione delle frequenze per l'intensità del colore
norm_freqs = top_subjects / top_subjects.max()

# Generazione di colori sfumati in base alla frequenza
colors = [plt.get_cmap('viridis')(intensity) for intensity in norm_freqs * 0.75 + 0.25]  # Assicura una certa intensità minima

# Creazione del diagramma a barre
plt.figure(figsize=(10, 8))  # Dimensione aggiustata per migliorare la leggibilità
sns.barplot(x=top_subjects.values, y=top_subjects.index, palette=colors)  # Utilizzo di seaborn per una migliore personalizzazione
plt.title('Top 20 Most Frequent Subjects in Dataframe Train')
plt.xlabel('Frequency')
plt.ylabel('Subjects')
plt.tight_layout()  # Aggiunge spazio intorno al grafico per evitare sovrapposizioni
plt.show()



### Knowledge Base

In [ ]:
# Crea un subset con solo le righe che hanno label 'true'
knowledge_base = df_train[df_train['label'] == 'true']
knowledge_base.info()

In [ ]:
print("Valori unici in 'subjects':")
print(knowledge_base['subjects'].nunique())
print(knowledge_base['subjects'].unique())

In [ ]:
# Rimuovere gli spazi bianchi all'inizio e alla fine di ogni stringa e dividere per virgola
subjects_series = knowledge_base['subjects'].dropna().str.strip().str.split(',')

# Pulire ogni elemento all'interno delle liste rimuovendo spazi extra
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])

# Aggiungere un passaggio per sostituire le liste vuote con un valore che indica l'assenza di soggetti
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)

# "Esplodere" la serie per avere ogni soggetto come una riga separata
exploded_subjects = subjects_series.explode()

# Se ci sono state liste vuote, ora saranno NaN, quindi sostituire NaN con un placeholder
# Se vuoi contare questi come una categoria separata, puoi sostituirli con una stringa specifica
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare le frequenze, inclusi i valori placeholder per le righe vuote
subject_frequencies = exploded_subjects.value_counts()

# Stampare le frequenze, che ora includono anche il conteggio per 'No Subject'
print(subject_frequencies)
print(len(subject_frequencies))


In [ ]:
# Pulizia iniziale dei dati, inclusa la gestione dei soggetti vuoti
subjects_series = knowledge_base['subjects'].dropna().str.strip().str.split(',')
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)
exploded_subjects = subjects_series.explode()
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare la frequenza di ciascun soggetto, inclusi quelli etichettati come 'No Subject'
subject_counts = exploded_subjects.value_counts()

# Selezionare i primi 20 soggetti più frequenti
top_subjects = subject_counts.head(20)

# Normalizzazione delle frequenze per l'intensità del colore
norm_freqs = top_subjects / top_subjects.max()

# Generazione di colori sfumati in base alla frequenza
colors = [plt.get_cmap('viridis')(intensity) for intensity in norm_freqs * 0.75 + 0.25]  # Assicura una certa intensità minima

# Creazione del diagramma a barre
plt.figure(figsize=(10, 8))  # Dimensione aggiustata per migliorare la leggibilità
sns.barplot(x=top_subjects.values, y=top_subjects.index, palette=colors)  # Utilizzo di seaborn per una migliore personalizzazione
plt.title('Top 20 Most Frequent Subjects in Knowledge Base')
plt.xlabel('Frequency')
plt.ylabel('Subjects')
plt.tight_layout()  # Aggiunge spazio intorno al grafico per evitare sovrapposizioni
plt.show()

In [ ]:
def plot_top_words(text_data, n=20, base_color='blue'):
    vec = CountVectorizer(stop_words='english').fit(text_data)
    bag_of_words = vec.transform(text_data)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]
    words, freqs = zip(*words_freq)
    
    # Normalizzazione delle frequenze per l'intensità del colore
    norm_freqs = np.array(freqs) / max(freqs)
    
    # Generazione di colori sfumati in base alla frequenza
    colors = [plt.get_cmap('Blues')(intensity) for intensity in norm_freqs * 0.75 + 0.25]  # Assicura una certa intensità minima
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x=list(freqs), y=list(words), palette=colors)  # Utilizzo di 'palette' al posto di 'color'
    plt.title('Top 20 Most Frequent Words in Knowledge Base')
    plt.xlabel('Frequency')
    plt.ylabel('Words in Claim')
    plt.show()

# Applica la funzione a tutte le 'Claim' nel dataset, con sfumatura basata sulla frequenza
all_claims = knowledge_base['claim']
plot_top_words(all_claims, base_color='blue')  # 'base_color' non è più necessario ma mantenuto per coerenza

### Test Dataset


In [ ]:
df_test = pd.read_csv(r'C:\Users\c.farallo\OneDrive - BE THINK, SOLVE, EXECUTE S.P.A\Desktop\Thesis\PUBHEALTH\test.tsv', sep='\t')
df_test.loc[1:3]

In [ ]:
print("Valori unici in 'label':")
print(df_test['label'].nunique())
print(df_test['label'].unique())

In [ ]:
print("Valori unici in 'subjects':")
print(df_test['subjects'].nunique())
print(df_test['subjects'].unique())
true_label_count = df_test[df_test['label'] == 'true'].shape[0]
print(f"Numero di righe con label 'true': {true_label_count}")

In [ ]:
# Prima, convertiamo la colonna 'subjects' in una serie di liste di subjects
subjects_series = df_test['subjects'].dropna().str.split(',')

# Ora, "esplodiamo" questa serie in modo che ogni elemento di ogni lista diventi una riga
exploded_subjects = subjects_series.explode()

# Calcoliamo la frequenza di ogni subject unico
subject_frequencies = exploded_subjects.value_counts()

# Stampiamo le frequenze
print(subject_frequencies)

In [ ]:
# Rimuovere gli spazi bianchi all'inizio e alla fine di ogni stringa e dividere per virgola
subjects_series = df_test['subjects'].dropna().str.strip().str.split(',')

# Pulire ogni elemento all'interno delle liste rimuovendo spazi extra
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])

# Aggiungere un passaggio per sostituire le liste vuote con un valore che indica l'assenza di soggetti
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)

# "Esplodere" la serie per avere ogni soggetto come una riga separata
exploded_subjects = subjects_series.explode()

# Se ci sono state liste vuote, ora saranno NaN, quindi sostituire NaN con un placeholder
# Se vuoi contare questi come una categoria separata, puoi sostituirli con una stringa specifica
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare le frequenze, inclusi i valori placeholder per le righe vuote
subject_frequencies = exploded_subjects.value_counts()

# Stampare le frequenze, che ora includono anche il conteggio per 'No Subject'
print(subject_frequencies)
print(len(subject_frequencies))

In [ ]:
# Pulizia iniziale dei dati, inclusa la gestione dei soggetti vuoti
subjects_series = df_test['subjects'].dropna().str.strip().str.split(',')
subjects_series = subjects_series.apply(lambda x: [i.strip() for i in x if i.strip() != ''])
subjects_series = subjects_series.apply(lambda x: np.nan if len(x) == 0 else x)
exploded_subjects = subjects_series.explode()
exploded_subjects = exploded_subjects.fillna('No Subject')

# Calcolare la frequenza di ciascun soggetto, inclusi quelli etichettati come 'No Subject'
subject_counts = exploded_subjects.value_counts()

# Selezionare i primi 20 soggetti più frequenti
top_subjects = subject_counts.head(20)

# Normalizzazione delle frequenze per l'intensità del colore
norm_freqs = top_subjects / top_subjects.max()

# Generazione di colori sfumati in base alla frequenza
colors = [plt.get_cmap('viridis')(intensity) for intensity in norm_freqs * 0.75 + 0.25]  # Assicura una certa intensità minima

# Creazione del diagramma a barre
plt.figure(figsize=(10, 8))  # Dimensione aggiustata per migliorare la leggibilità
sns.barplot(x=top_subjects.values, y=top_subjects.index, palette=colors)  # Utilizzo di seaborn per una migliore personalizzazione
plt.title('Top 20 Most Frequent Subjects in Dataframe Test')
plt.xlabel('Frequency')
plt.ylabel('Subjects')
plt.tight_layout()  # Aggiunge spazio intorno al grafico per evitare sovrapposizioni
plt.show()

In [ ]:
def plot_top_words(text_data, n=20, base_color='blue'):
    vec = CountVectorizer(stop_words='english').fit(text_data)
    bag_of_words = vec.transform(text_data)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]
    words, freqs = zip(*words_freq)
    
    # Normalizzazione delle frequenze per l'intensità del colore
    norm_freqs = np.array(freqs) / max(freqs)
    
    # Generazione di colori sfumati in base alla frequenza
    colors = [plt.get_cmap('Blues')(intensity) for intensity in norm_freqs * 0.75 + 0.25]  # Assicura una certa intensità minima
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x=list(freqs), y=list(words), palette=colors)  # Utilizzo di 'palette' al posto di 'color'
    plt.title('Top 20 Most Frequent Words in Dataframe Test')
    plt.xlabel('Frequency')
    plt.ylabel('Words in Claim')
    plt.show()

# Applica la funzione a tutte le 'Claim' nel dataset, con sfumatura basata sulla frequenza
all_claims = df_test['claim']
plot_top_words(all_claims, base_color='blue')  # 'base_color' non è più necessario ma mantenuto per coerenza

## Save Files

* Al momento non c'è nessun pre-processing e l'esplorazione è limitata; prendiamo solo i dataset che ci servono.

* Prendiamo solo le colonne che ci serviranno per il progetto

In [ ]:
#knowledge_base.to_csv('knowledge_base')
#df_test.to_csv('df_test')